In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print("Text Length: {}".format(len(text)))

Text Length: 1115394


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char, _ in zip(char2idx, range(20)):
    print('    {:4s}: {:3d},'.format(repr(char),char2idx[char]))
print('    ...\n}')

{
    '\n':   0,
    ' ' :   1,
    '!' :   2,
    '$' :   3,
    '&' :   4,
    "'" :   5,
    ',' :   6,
    '-' :   7,
    '.' :   8,
    '3' :   9,
    ':' :  10,
    ';' :  11,
    '?' :  12,
    'A' :  13,
    'B' :  14,
    'C' :  15,
    'D' :  16,
    'E' :  17,
    'F' :  18,
    'G' :  19,
    ...
}


In [8]:
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
seq_length = 100
examples_per_epoch = len(text) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in dataset.take(1):
    print('Input Data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target Data: ', repr(''.join(idx2char[target_example.numpy()])))

Input Data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target Data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print('{:4d}step'.format(i))
    print("    Input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("    Expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0step
    Input: 18 ('F')
    Expected output: 47 ('i')
   1step
    Input: 47 ('i')
    Expected output: 56 ('r')
   2step
    Input: 56 ('r')
    Expected output: 57 ('s')
   3step
    Input: 57 ('s')
    Expected output: 58 ('t')
   4step
    Input: 58 ('t')
    Expected output: 1 (' ')


In [14]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [15]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [18]:
with tf.device('cpu:0'):
    for input_example_batch, target_example_batch in dataset.take(1):
        example_batch_predictions = model(input_example_batch)
        print(example_batch_predictions.shape, "# (Batch size, Sequence length, vocab size)")

(64, 100, 65) # (Batch size, Sequence length, vocab size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([59, 61, 13, 34, 38,  2, 57, 51, 20, 12, 11, 33,  6,  7, 58,  1,  4,
        9, 10, 31, 54, 59, 26, 12, 39, 55,  6, 28, 45, 42, 60, 44, 30,  1,
       21, 25, 53, 55, 46, 14, 34, 28, 53, 60, 57, 30, 22, 37, 24, 56, 61,
        9, 24, 25, 53, 32,  0, 63, 19, 28,  3, 39, 60, 63,  0,  4, 63,  5,
       47, 11, 19, 19, 64, 42, 21, 15, 43, 56, 20,  6, 56, 50, 33, 46, 25,
       46, 40, 59, 46, 16, 32, 45, 46, 40, 15, 27, 58, 40, 11, 41],
      dtype=int64)

In [22]:
print('Input: \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Predicted next char: \n', repr(''.join(idx2char[sampled_indices])))

Input: 
 ',\nInfusing him with self and vain conceit,\nAs if this flesh which walls about our life,\nWere brass i'

Predicted next char: 
 "uwAVZ!smH?;U,-t &3:SpuN?aq,PgdvfR IMoqhBVPovsRJYLrw3LMoT\nyGP$avy\n&y'i;GGzdICerH,rlUhMhbuhDTghbCOtb;c"


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print('Predicted array shape: ', example_batch_predictions.shape, '#batch size, sequence length, vocab sizse')
print('Scalar Loss:           ', example_batch_loss.numpy().mean())

Predicted array shape:  (64, 100, 65) #batch size, sequence length, vocab sizse
Scalar Loss:            4.17377


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only=True
)

In [26]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 24s 138ms/step - loss: 2.5729
Epoch 2/10
172/172 [==============================] - 24s 140ms/step - loss: 1.8798
Epoch 3/10
172/172 [==============================] - 24s 142ms/step - loss: 1.6311
Epoch 4/10
172/172 [==============================] - 24s 142ms/step - loss: 1.4985
Epoch 5/10
172/172 [==============================] - 24s 141ms/step - loss: 1.4195
Epoch 6/10
172/172 [==============================] - 24s 141ms/step - loss: 1.3625
Epoch 7/10
172/172 [==============================] - 24s 141ms/step - loss: 1.3167
Epoch 8/10
172/172 [==============================] - 24s 141ms/step - loss: 1.2773
Epoch 9/10
172/172 [==============================] - 24s 141ms/step - loss: 1.2410
Epoch 10/10
172/172 [==============================] - 24s 142ms/step - loss: 1.2052


In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [32]:
start_string=u'ROMEO: '
input_eval= [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)
input_eval

<tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[30, 27, 25, 17, 27, 10,  1]])>

In [35]:
predictions = model(input_eval)
predictions.shape

TensorShape([1, 7, 65])

In [37]:
predictions = tf.squeeze(predictions,0)
predictions.shape

TensorShape([7, 65])

In [50]:
predicted_id = tf.random.categorical(predictions, num_samples=1)
print(predicted_id)
predicted_id = predicted_id[-1,0].numpy()
print(predicted_id)

tf.Tensor(
[[27]
 [26]
 [26]
 [26]
 [26]
 [26]
 [24]], shape=(7, 1), dtype=int64)
24


In [52]:
input_eval = tf.expand_dims([predicted_id], 0)
model(input_eval)

<tf.Tensor: shape=(1, 1, 65), dtype=float32, numpy=
array([[[-1.7272413 , -1.2574241 , -0.72170895,  0.71675956,
          0.60398984, -1.8925035 , -0.9863711 , -3.1031492 ,
         -1.4229251 ,  1.8642952 ,  1.3056587 , -1.4313487 ,
         -0.9156358 ,  6.4796343 ,  3.976162  ,  2.4540565 ,
          2.5839133 ,  5.8690267 ,  2.417949  ,  3.4565592 ,
          1.4969034 ,  5.0612116 ,  1.0864896 ,  2.0171728 ,
          3.6579578 ,  4.17151   ,  3.4925382 ,  5.2435865 ,
          0.87655765,  0.88971865,  3.6698987 ,  4.727582  ,
          1.4713666 ,  5.5417194 ,  1.4050274 ,  2.9742107 ,
          1.2784101 ,  5.0737596 ,  1.2665923 ,  0.72328067,
         -2.3066692 , -5.9606366 , -0.6483144 ,  0.12849848,
         -3.7022803 , -4.303837  , -3.580127  ,  0.4032501 ,
         -1.810353  , -2.556422  , -1.2360313 , -2.5044863 ,
         -3.708578  ,  3.0261848 , -2.54353   , -2.5371032 ,
         -1.7329506 , -4.189733  , -5.266301  ,  2.124294  ,
         -1.9931201 , -2.6665618 

In [55]:
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = 1.0
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [56]:
result = generate_text(model, start_string=u'ROMEO: ')
print(len(result))
print(result)


1007
ROMEO: KTARS:

NCATKA:
Ay, die these knees whose courties's bosom
A noble heer; when I am call'd would
your accurterch! This laudful shame mngot of
these houses. Sag me; follows.

CAPULET:
Cut them for you: for I come all in that:
Therefore how sast your son, sir; O
Pompey! friar, I fear, to take thy credent I hither
indeed, and mine or kind and noise a lady's blood,
With many execution of my foesween I am no talk.

CAMILLO:
I am the report, suse me, wife Richard Clarence, down my lord?

KING RICHARD II:
That had she serve him of the chastial pifit him;
Resquaching lights, back; and it is less
ere you would dissembling stirrs to my true.
Wilt thither where unendured men
You ploud me to none entreaty
Amen struck then a doven;
And, and abuse my call, lady, and femplars,
A measure farewell mercy with some head
And that bear me will not that: you joy himself?
'Tis said, my lord, werrick, sir, respected in this worst, hail,
Upon my blood, the pitiful undown dissemple
Towerhold all of h

In [137]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size = BATCH_SIZE
)

In [138]:
optimizer = tf.keras.optimizers.Adam()

In [139]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True
            )
        )
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    return loss

In [140]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = model.reset_states()
    
    for(batch_n, (inp,target)) in enumerate(dataset):
        loss = train_step(inp, target)
        
        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))
            
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
        
    print('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print('{} seconds per 1 Epoch'.format(time.time() - start))
    
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174795150756836
Epoch 1 Batch 100 Loss 2.3201682567596436
Epoch 1 Loss 2.0595
28.870805501937866 seconds per 1 Epoch
Epoch 2 Batch 0 Loss 2.0655455589294434
Epoch 2 Batch 100 Loss 1.8349605798721313
Epoch 2 Loss 1.7293
26.76218557357788 seconds per 1 Epoch
Epoch 3 Batch 0 Loss 1.7068895101547241
Epoch 3 Batch 100 Loss 1.5943655967712402
Epoch 3 Loss 1.5465
26.854443550109863 seconds per 1 Epoch
Epoch 4 Batch 0 Loss 1.5381665229797363
Epoch 4 Batch 100 Loss 1.4727193117141724
Epoch 4 Loss 1.4816
26.8249990940094 seconds per 1 Epoch
Epoch 5 Batch 0 Loss 1.4253158569335938
Epoch 5 Batch 100 Loss 1.4020183086395264
Epoch 5 Loss 1.3890
27.07475519180298 seconds per 1 Epoch
Epoch 6 Batch 0 Loss 1.3564867973327637
Epoch 6 Batch 100 Loss 1.3611595630645752
Epoch 6 Loss 1.3321
26.7163143157959 seconds per 1 Epoch
Epoch 7 Batch 0 Loss 1.2596545219421387
Epoch 7 Batch 100 Loss 1.2989153861999512
Epoch 7 Loss 1.3377
26.91620707511902 seconds per 1 Epoch
Epoch 8 Batch 0 Loss 